# 5.1. 层和块

块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 使用块进行抽象的一个好处是可以将一些块组合成更大的组件， 这一过程通常是递归的

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2324,  0.1361,  0.1700,  0.0930,  0.1251, -0.0559, -0.0389, -0.1248,
          0.0947, -0.1072],
        [-0.0955, -0.0936,  0.0952,  0.0572,  0.1403, -0.1299, -0.0677,  0.0167,
          0.1634, -0.0623]], grad_fn=<AddmmBackward>)

# 5.1.1. 自定义块

每个块必须提供的基本功能:

1.将输入数据作为其前向传播函数的参数。

2.通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

4.存储和访问前向传播计算所需的参数。

5.根据需要初始化模型参数。

In [3]:
# 我们从零开始编写一个块。 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层
# 实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数

class MLP(nn.Module):
    # 调用MLP的父类Module的构造函数来执行必要的初始化。
    # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    # forward前向传播
    def forward(self, X):
        # nn.functional模块中的ReLU函数: F.relu()
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0050,  0.0063,  0.2533, -0.0030,  0.0796,  0.1360,  0.1707, -0.1264,
         -0.0117, -0.1887],
        [ 0.0255,  0.0279,  0.2426, -0.0208,  0.2265,  0.1343,  0.1802, -0.1370,
         -0.0349, -0.1517]], grad_fn=<AddmmBackward>)

# 5.1.2. 顺序块

为了把其他模块串起来， 构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

1.一种将块逐个追加到列表中的函数；

2.一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():   # values()函数返回一个字典中所有的值
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0247,  0.2268,  0.1256, -0.1297,  0.1407,  0.0567, -0.1858,  0.0464,
         -0.0541, -0.1683],
        [-0.0014,  0.0918,  0.1577, -0.1118,  0.0540,  0.1272, -0.1133,  0.0212,
         -0.0733, -0.0902]], grad_fn=<AddmmBackward>)

# 5.1.3. 在前向传播函数中执行代码

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(0.1726, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))


In [13]:
chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)

chimera(X)

tensor(-0.1411, grad_fn=<SumBackward0>)

# 5.1.4. 效率

# 5.1.5. 小结

一个块可以由许多层组成；一个块可以由许多块组成。

块可以包含代码。

块负责大量的内部处理，包括参数初始化和反向传播。

层和块的顺序连接由Sequential块处理。

# 5.1.6. 练习